# Ejercicio Negative Transfer

El objetivo de este ejercicio es visualizar el efecto del Negative Transfer cuando se aplica Transfer Learning. Para ello, se debera entrenar una red neuronal, cuya arquitectura esta definida por la clase `ConvModel` de la celda que se encuentra a continuacion, utilizando el dataset ya conocido de imagenes de perros y gatos. Luego, se utilizara dicho modelo preentrenado para hacer transfer learning sobre un dataset de imagenes de frutas (banana y melon concretamente). Este dataset cuenta con 900 imagenes de cada clase en el set de entrenamiento, y 100 en el de testeo. 

Dada la poca relacion que hay entre imagenes de perros y gatos vs bananas y melones, se espera entonces que el entrenamiento utilizando Transfer Learning tenga resultados pobres, comparados con los obtenidos por el modelo base durante su entrenamiento.

Para probar esto se debe:

- Entrenar la arquitectura con el dataset de perros y gatos durante unas 40 o 50 epocas, aplicando Data Augmentation para evitar el sobreentrenamiento. Una vez que se logre un resultado satisfactorio (accuracy por encima de 75%), se puede guardar dicho modelo para utilizarlo como base para el Transfer Learning.
- Reemplazar la ultima capa del modelo preentrenado y re-entrenar utilizando el dataset de frutas. Realizar el entrenamiento durante unas 20 epocas, utilizando un learning rate menor al del punto anterior. Observar los resultados obtenidos y compararlos con los del punto anterior.
- Por ultimo, entrenar la misma arquitectura, desde cero, utilizando el dataset de frutas. Observar los resultados obtenidos y compararlos con los del punto anterior.

In [1]:
!pip install torchmetrics
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 7.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os

import torch
import torchvision
import torchsummary
import torchmetrics
from torch.utils.tensorboard import SummaryWriter

### Descargar datasets de perros y gatos y de frutas

In [3]:
# Descarga dataset de perros y gatos
!gdown 1bx73wWKNgxFZ6sSFSeaKBVcuGtdraKh-
!unzip perros_y_gatos.zip > /dev/null

# Descarga dataset de frutas
!gdown 1S2bnSwFrV3GKjGaLs7Br7_0iIhXLR_us
!unzip frutas.zip > /dev/null

Downloading...
From: https://drive.google.com/uc?id=1bx73wWKNgxFZ6sSFSeaKBVcuGtdraKh-
To: /content/perros_y_gatos.zip
100% 90.8M/90.8M [00:02<00:00, 41.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1S2bnSwFrV3GKjGaLs7Br7_0iIhXLR_us
To: /content/frutas.zip
100% 75.8M/75.8M [00:02<00:00, 34.4MB/s]


### Función de entrenamiento

In [ ]:
def train(model, optimizer, criterion, metric, data, epochs, tb_writer=None):

    train_loader = data["train"]
    valid_loader = data["valid"]

    train_writer = tb_writer["train"]
    valid_writer = tb_writer["valid"]

    if tb_writer:
        train_writer.add_graph(model, torch.zeros((1, 3, data["image_width"], data["image_height"])))
        valid_writer.add_graph(model, torch.zeros((1, 3, data["image_width"], data["image_height"])))

    if torch.cuda.is_available():
        model.to("cuda")
        metric.to("cuda")

    train_loss = []
    train_acc = []
    valid_loss = []
    valid_acc = []

    for epoch in range(epochs):

        # Pongo el modelo en modo entrenamiento
        model.train()

        epoch_train_loss = 0.0
        epoch_train_accuracy = 0.0

        for train_data, train_target in train_loader:
            
            if torch.cuda.is_available():
                train_data = train_data.to("cuda")
                train_target = train_target.to("cuda")

            optimizer.zero_grad()
            output = model(train_data.float())
            loss = criterion(output, train_target)
            epoch_train_loss += loss.item()
            loss.backward()
            optimizer.step()

            accuracy = metric(output, train_target)
            epoch_train_accuracy += accuracy.item()

        epoch_train_loss = epoch_train_loss / len(train_loader)
        epoch_train_accuracy = epoch_train_accuracy / len(train_loader)
        train_loss.append(epoch_train_loss)
        train_acc.append(epoch_train_accuracy)

        # Pongo el modelo en modo testeo
        model.eval()

        epoch_valid_loss = 0.0
        epoch_valid_accuracy = 0.0

        for valid_data, valid_target in valid_loader:
            if torch.cuda.is_available():
                valid_data = valid_data.to("cuda")
                valid_target = valid_target.to("cuda")

            output = model(valid_data.float())
            epoch_valid_loss += criterion(output, valid_target).item()
            epoch_valid_accuracy += metric(output, valid_target).item()
            
        epoch_valid_loss = epoch_valid_loss / len(valid_loader)
        epoch_valid_accuracy = epoch_valid_accuracy / len(valid_loader)
        valid_loss.append(epoch_valid_loss)
        valid_acc.append(epoch_valid_accuracy)

        print("Epoch: {}/{} - Train loss {:.6f} - Train Accuracy {:.6f} - Valid Loss {:.6f} - Valid Accuracy {:.6f}".format(
        epoch+1, epochs, epoch_train_loss, epoch_train_accuracy, epoch_valid_loss, epoch_valid_accuracy))

        if tb_writer:
            train_writer.add_scalar("loss", epoch_train_loss, epoch)
            valid_writer.add_scalar("loss", epoch_valid_loss, epoch)
            train_writer.add_scalar("accuracy", epoch_train_accuracy, epoch)
            valid_writer.add_scalar("accuracy", epoch_valid_accuracy, epoch)
            train_writer.flush()
            valid_writer.flush()

    history = {}
    history["train_loss"] = train_loss
    history["train_acc"] = train_acc
    history["valid_loss"] = valid_loss
    history["valid_acc"] = valid_acc

    return history

### Arquitectura de red neuronal

In [ ]:
CANTIDAD_CLASES = 2
ANCHO_IMAGENES = 150
ALTO_IMAGENES = 150

In [ ]:
class ConvModel(torch.nn.Module):
    def __init__(self, output_units):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding='same')
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding='same')
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding='same')
        self.pool3 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv4 = torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding='same')
        self.pool4 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = torch.nn.Linear(in_features=10368, out_features=256)
        self.fc2 = torch.nn.Linear(in_features=256, out_features=output_units)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = self.pool3(torch.relu(self.conv3(x)))
        x = self.pool4(torch.relu(self.conv4(x)))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

conv_model = ConvModel(CANTIDAD_CLASES)

# Si hay una GPU disponible muevo el modelo allí para aprovechar ese recurso
if torch.cuda.is_available():
    conv_model.to("cuda")

torchsummary.summary(conv_model, (3, ANCHO_IMAGENES, ALTO_IMAGENES))